Notice that:
- best start: set m = 5000 and n = 5 in _quantreg.py
- random start: set m = 1 and n = 1 instead

In [ ]:
import sys
sys.path.insert(0, '..')

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from caviar import CaviarModel
from var_tests import binomial_test, kupiec_pof_test, christoffersen_test, traffic_light_test

In [ ]:
while True:
    data = yf.download('SPY', start='2001-01-01')
    if data.shape[0] > 0:
        break

In [ ]:
log_returns = (data['Close'].apply(np.log).diff() * 100).dropna()
in_samples = log_returns[:'2015']
out_samples = log_returns['2016':]

In [ ]:
hit_rate = lambda ret, var: np.mean(ret < var)

In [ ]:
quantiles = [.05, .01]
methods = ['numeric']
models = ['adaptive', 'symmetric', 'asymmetric', 'igarch']

In [ ]:
stats_df = []
beta_stats_df = []

for q in quantiles:
    for model in models:
        for method in methods:
            print(f'Runing quantile: {q}, model: {model}, method: {method}')
            for i in range(30):
                caviar_model = CaviarModel(q, model, method)
                caviar_model.fit(in_samples)
                
                # as the last observation is the VaR forecast
                in_VaR = caviar_model.predict(in_samples, 'in')[:-1]
                out_VaR = caviar_model.predict(out_samples, 'out')[:-1]
                
                # statistics
                stat = {
                    'quantile': q,
                    'model': model,
                    'method': method,
                    'trial': i,
                    'loss': caviar_model.training_loss,
                    'hit_rate_in': hit_rate(in_samples, in_VaR),
                    'hit_rate_out': hit_rate(out_samples, out_VaR),
                    'dq_in': caviar_model.dq_test(in_samples, 'in'),
                    'dq_out': caviar_model.dq_test(out_samples, 'out'),
                    'binom_in': binomial_test(in_samples, in_VaR, q),
                    'binom_out': binomial_test(out_samples, out_VaR, q),
                    'traffic_in': traffic_light_test(in_samples, in_VaR, q)[0],
                    'traffic_out': traffic_light_test(out_samples, out_VaR, q)[0],
                    'kupiec_in': kupiec_pof_test(in_samples, in_VaR, q),
                    'kupiec_out': kupiec_pof_test(out_samples, out_VaR, q),
                    'independent_in': christoffersen_test(in_samples, in_VaR),
                    'independent_out': christoffersen_test(out_samples, out_VaR)
                }
                
                beta_stat = caviar_model.beta_summary()
                beta_stat['quantile'] = q
                beta_stat['model'] = model
                beta_stat['method'] = method
                beta_stat['trial'] = i
                
                stats_df.append(stat)
                beta_stats_df.append(beta_stat)

In [ ]:
pd.DataFrame(stats_df)

In [ ]:
stats_df = pd.DataFrame(stats_df)
beta_stats_df = pd.concat(beta_stats_df)
beta_stats_df.index.name = 'beta'

In [ ]:
stats_df.to_csv('stat_RQ_best_start.csv', index=None)
beta_stats_df.to_csv('beta_RQ_best_start.csv')